In [29]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZARRAR\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
csv = pd.read_csv('spam.csv', encoding="ISO-8859-1")
df = pd.DataFrame(csv)#series

In [8]:
df.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 108.9+ KB


In [15]:
# check class distribution
classes = df[df.columns[0]]
classes.value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [21]:
# convert class labels to binary values, 0 = ham  1 = spam
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

# quick check
print('rno v1 Encoded')
for i in range(0,10):
    print(i,classes[i],'\t', Y[i])

rno v1 Encoded
0 ham 	 0
1 ham 	 0
2 spam 	 1
3 ham 	 0
4 ham 	 0
5 spam 	 1
6 ham 	 0
7 ham 	 0
8 spam 	 1
9 spam 	 1


In [22]:
# store SMS message data
text_messages = df[df.columns[1]]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: v2, dtype: object


In [32]:
# expressions can be found at https://regexlib.com/
# use regular expressions to replace email addresses, urls, phone numbers, etc.

# replace email addresses with 'emailaddr
processed = text_messages.str.replace(r'^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$', 'emailaddr')
# replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')
# replace money symbols with 'moneysymb'
processed = text_messages.str.replace(r'£|\$', 'moneysymb')
# replace 10 digit phone numbers with 'phonenum'
processed = text_messages.str.replace(r'^[2-9]\d{2}-\d{3}-\d{4}$', 'phonenum')

# replace normal numbers with 'num'
processed = text_messages.str.replace(r'\d+(\.\d+)?', 'num')

<ipython-input-32-ea7df8d1124e>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = text_messages.str.replace(r'^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$', 'emailaddr')
<ipython-input-32-ea7df8d1124e>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')
<ipython-input-32-ea7df8d1124e>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = text_messages.str.replace(r'£|\$', 'moneysymb')
<ipython-input-32-ea7df8d1124e>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = text_messages.str.replace(r'^[2-9]\d{2}-\d{3}-\d{4}$', 'phonenum')
<ipython-input-32-ea7df8d1124e>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = text

In [33]:
# remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

<ipython-input-33-0afdbb470505>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'[^\w\d\s]', ' ')
<ipython-input-33-0afdbb470505>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'\s+', ' ')
<ipython-input-33-0afdbb470505>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'^\s+|\s+?$', '')


In [34]:
# change all words to lower case
processed = processed.str.lower()
processed.head()

0    go until jurong point crazy available only in ...
1                              ok lar joking wif u oni
2    free entry in num a wkly comp to win fa cup fi...
3          u dun say so early hor u c already then say
4    nah i don t think he goes to usf he lives arou...
Name: v2, dtype: object

In [35]:
from nltk.corpus import stopwords

# remove stop words from text messages
# stop words are basically a set of commonly used words in any language such as i, me, to, it, etc.

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: 
        ' '.join(term for term in x.split() if term not in stop_words)
        )
processed.head()

0    go jurong point crazy available bugis n great ...
1                              ok lar joking wif u oni
2    free entry num wkly comp win fa cup final tkts...
3                  u dun say early hor u c already say
4               nah think goes usf lives around though
Name: v2, dtype: object

In [36]:
# remove word stems using a Porter stemmer
# stemming is the process of reducing a word to its word stem such as removing -ing
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: 
    ' '.join(ps.stem(term) for term in x.split()))

processed.head()

0    go jurong point crazi avail bugi n great world...
1                                ok lar joke wif u oni
2    free entri num wkli comp win fa cup final tkt ...
3                  u dun say earli hor u c alreadi say
4                 nah think goe usf live around though
Name: v2, dtype: object

In [37]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
# create bag-of-words
all_words = []
for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)

# FreqDist class is used to encode “frequency distributions”, which count the number of times that each outcome of an experiment occurs

all_words = nltk.FreqDist(all_words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ZARRAR\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [38]:
all_words

FreqDist({'num': 2959, 'u': 1192, 'call': 672, 'go': 453, 'get': 452, 'ur': 385, 'gt': 318, 'lt': 316, 'å': 303, 'come': 301, ...})

In [40]:
# print the total number of words and the 15 most common words
print(f'Number of words: {format(len(all_words))}')
print(f'Most common words: {format(all_words.most_common(10))}')

Number of words: 6534
Most common words: [('num', 2959), ('u', 1192), ('call', 672), ('go', 453), ('get', 452), ('ur', 385), ('gt', 318), ('lt', 316), ('å', 303), ('come', 301)]


In [42]:
# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]
word_features

['go',
 'jurong',
 'point',
 'crazi',
 'avail',
 'bugi',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'got',
 'amor',
 'wat',
 'ok',
 'lar',
 'joke',
 'wif',
 'u',
 'oni',
 'free',
 'entri',
 'num',
 'wkli',
 'comp',
 'win',
 'fa',
 'cup',
 'final',
 'tkt',
 'numst',
 'may',
 'text',
 'receiv',
 'question',
 'std',
 'txt',
 'rate',
 'c',
 'appli',
 'numovernum',
 'dun',
 'say',
 'earli',
 'hor',
 'alreadi',
 'nah',
 'think',
 'goe',
 'usf',
 'live',
 'around',
 'though',
 'freemsg',
 'hey',
 'darl',
 'week',
 'word',
 'back',
 'like',
 'fun',
 'still',
 'tb',
 'xxx',
 'chg',
 'send',
 'å',
 'rcv',
 'even',
 'brother',
 'speak',
 'treat',
 'aid',
 'patent',
 'per',
 'request',
 'mell',
 'oru',
 'minnaminungint',
 'nurungu',
 'vettam',
 'set',
 'callertun',
 'caller',
 'press',
 'copi',
 'friend',
 'winner',
 'valu',
 'network',
 'custom',
 'select',
 'receivea',
 'prize',
 'reward',
 'claim',
 'call',
 'code',
 'klnum',
 'valid',
 'hour',
 'mobil',
 'month',
 'r',
 'entit

In [56]:
# find_features function will determine which of the 1500 word features are contained in the email/message
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
    return features
# example
print('processed[0]:',processed[0])
features = find_features(processed[0])
print('features at 0th index:',list(features.items())[:10])
for key, value in features.items():
    if value == True:
        print(key)

processed[0]: go jurong point crazi avail bugi n great world la e buffet cine got amor wat
features at 0th index: [('go', True), ('jurong', True), ('point', True), ('crazi', True), ('avail', True), ('bugi', True), ('n', True), ('great', True), ('world', True), ('la', True)]
go
jurong
point
crazi
avail
bugi
n
great
world
la
e
buffet
cine
got
amor
wat


In [57]:
# do it for all the messages
messages = list(zip(processed, Y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]

In [58]:
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [59]:
print('Training:',len(training))
print('Testing:',len(testing))

Training: 4179
Testing: 1393


In [60]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'linear'))

# train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print("SVC Accuracy: {}".format(accuracy))

SVC Accuracy: 98.34888729361091


In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression"]
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression()
]

models = zip(names, classifiers)
for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 94.18521177315147
Decision Tree Accuracy: 97.20028715003589
Random Forest Accuracy: 98.34888729361091
Logistic Regression Accuracy: 98.56424982053123


In [62]:
# ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression()
]
models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 98.56424982053123


In [63]:
# class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [65]:
# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']]
    )

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1196
           1       1.00      0.87      0.93       197

    accuracy                           0.98      1393
   macro avg       0.99      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393



predicted     
                  ham spam
actual ham       1196    0
       spam        25  172